# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
# Drop column with only one value
columns_list = df.columns.tolist()
columns_to_drop = [column for column in columns_list if len(df[column].unique())<2]
df_clean = df.drop(columns_to_drop, axis=1)

In [6]:
# Make a copy of df for encoding
df_encoded = df_clean.copy()

In [7]:
# Find categorical columns except issue_d and next_pymnt_d
categorical_feature_mask = df_encoded.dtypes==object
categorical_cols = df_encoded.columns[categorical_feature_mask].tolist()
categorical_cols.remove('issue_d')
categorical_cols.remove('next_pymnt_d')
categorical_cols

['home_ownership',
 'verification_status',
 'loan_status',
 'initial_list_status',
 'application_type']

In [8]:
# Custom Encoding for issue_d and next_pymnt_d
month_num = {   
   "Jan-2019": 1,
   "Feb-2019": 2,
   "Mar-2019": 3,
   "Apr-2019": 4,
   "May-2019": 5,
   "Jun-2019": 6,
   "Jul-2019": 7,
   "Aug-2019": 8,
   "Sep-2019": 9,
   "Oct-2019": 10,
   "Nov-2019": 11,
   "Dec-2019": 12,}

df_encoded["issue_d"] = df_encoded["issue_d"].apply(lambda x: month_num[x])
df_encoded["next_pymnt_d"] = df_encoded["next_pymnt_d"].apply(lambda x: month_num[x])

In [9]:
# Import labelencoder
from sklearn.preprocessing import LabelEncoder
# Instantiate labelencoder object
le = LabelEncoder()

# Apply le on categorical feature columns
df_encoded[categorical_cols] = df_encoded[categorical_cols].apply(lambda col: le.fit_transform(col))
df_encoded.head(10)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,dti,delinq_2yrs,...,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,10500.0,0.1719,375.35,3,66000.0,1,3,1,27.24,0.0,...,8.0,0.0,3.0,85.7,100.0,0.0,65687.0,38199.0,2000.0,61987.0
1,25000.0,0.2000,929.09,1,105000.0,2,3,1,20.23,0.0,...,17.0,0.0,6.0,91.2,50.0,1.0,271427.0,60641.0,41200.0,49197.0
2,20000.0,0.2000,529.88,1,56000.0,2,3,1,24.26,0.0,...,8.0,0.0,4.0,66.7,50.0,0.0,60644.0,45684.0,7500.0,43144.0
3,10000.0,0.1640,353.55,3,92000.0,2,3,1,31.44,0.0,...,10.0,0.0,1.0,100.0,50.0,1.0,99506.0,68784.0,19700.0,76506.0
4,22000.0,0.1474,520.39,1,52000.0,0,3,1,18.76,0.0,...,14.0,0.0,1.0,100.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0
5,25000.0,0.1797,634.43,1,90000.0,1,3,1,18.49,1.0,...,14.0,0.0,1.0,93.1,66.7,0.0,550717.0,418934.0,18800.0,369051.0
6,20400.0,0.2000,540.48,3,51315.0,1,3,1,25.63,0.0,...,8.0,0.0,4.0,100.0,0.0,1.0,40023.0,23469.0,3500.0,27523.0
7,10000.0,0.1557,349.46,2,66000.0,1,3,1,9.00,0.0,...,11.0,0.0,3.0,100.0,16.7,0.0,29300.0,22377.0,8400.0,12500.0
8,14000.0,0.0881,443.96,3,45000.0,0,3,1,10.24,0.0,...,9.0,0.0,4.0,100.0,0.0,0.0,25100.0,9724.0,12100.0,3000.0
9,10000.0,0.1474,345.39,3,63295.0,2,3,1,39.34,0.0,...,27.0,0.0,3.0,97.2,71.4,0.0,119822.0,102924.0,15300.0,80722.0


# Split the Data into Training and Testing

In [10]:
# Create our features
X = df_encoded.drop('loan_status', axis=1)

# Create our target
y = df_encoded['loan_status']

In [11]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,dti,delinq_2yrs,inq_last_6mths,...,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,1.726172,21.778153,0.217766,0.497697,...,12.571995,0.052138,2.219423,95.057627,30.626217,0.125972,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.743862,20.199244,0.718367,0.758122,...,6.021318,0.390633,1.897432,8.326426,33.631463,0.336732,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,1.000000,0.000000,0.000000,0.000000,...,2.000000,0.000000,0.000000,20.000000,0.000000,0.000000,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,1.000000,13.890000,0.000000,0.000000,...,8.000000,0.000000,1.000000,93.000000,0.000000,0.000000,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,2.000000,19.760000,0.000000,0.000000,...,11.000000,0.000000,2.000000,100.000000,20.000000,0.000000,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,2.000000,26.660000,0.000000,1.000000,...,16.000000,0.000000,3.000000,100.000000,50.000000,0.000000,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,3.000000,999.000000,18.000000,5.000000,...,72.000000,18.000000,19.000000,100.000000,100.000000,4.000000,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [12]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [13]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [14]:
Counter(y_train)

Counter({1: 51366, 0: 246})

In [15]:
Counter(y_test)

Counter({1: 17104, 0: 101})

In [16]:
balanced_acc_score = {}

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [17]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({1: 51366, 0: 51366})

In [18]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm_ros_df = pd.DataFrame(cm, index=['Actual 0', 'Actual 1'],
                    columns=['Predicted 0', 'Predicted 1'])
display(cm_ros_df)

,Predicted 0,Predicted 1
Actual 0,69,32
Actual 1,6000,11104


In [20]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_acc_score['Naive Random Oversampling'] = balanced_accuracy_score(y_test, y_pred)
balanced_acc_score

{'Naive Random Oversampling': 0.6661865905954487}

In [21]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print("Imbalanced Classification Report for Naive Random Oversampling")
print(classification_report_imbalanced(y_test, y_pred))

Imbalanced Classification Report for Naive Random Oversampling
                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.68      0.65      0.02      0.67      0.45       101
          1       1.00      0.65      0.68      0.79      0.67      0.44     17104

avg / total       0.99      0.65      0.68      0.78      0.67      0.44     17205



### SMOTE Oversampling

In [22]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)

In [23]:
Counter(y_resampled)

Counter({1: 51366, 0: 51366})

In [24]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [25]:
# Calculated the balanced accuracy score
balanced_acc_score['SMOTE Oversampling'] = balanced_accuracy_score(y_test, y_pred)
balanced_acc_score

{'Naive Random Oversampling': 0.6661865905954487,
 'SMOTE Oversampling': 0.6610112046050256}

In [26]:
# Display the confusion matrix
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm_smote_df = pd.DataFrame(cm, index=['Actual 0', 'Actual 1'],
                    columns=['Predicted 0', 'Predicted 1'])
display(cm_smote_df)

,Predicted 0,Predicted 1
Actual 0,66,35
Actual 1,5669,11435


In [27]:
# Print the imbalanced classification report
print("Imbalanced Classification Report for SMOTE Oversampling")
print(classification_report_imbalanced(y_test, y_pred))

Imbalanced Classification Report for SMOTE Oversampling
                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.65      0.67      0.02      0.66      0.44       101
          1       1.00      0.67      0.65      0.80      0.66      0.44     17104

avg / total       0.99      0.67      0.65      0.80      0.66      0.44     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [28]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 246, 1: 246})

In [29]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [30]:
# Calculated the balanced accuracy score
balanced_acc_score['ClusterCentroids Undersampling'] = balanced_accuracy_score(y_test, y_pred)
balanced_acc_score

{'Naive Random Oversampling': 0.6661865905954487,
 'SMOTE Oversampling': 0.6610112046050256,
 'ClusterCentroids Undersampling': 0.5349689494206671}

In [31]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_cc_df = pd.DataFrame(cm, index=['Actual 0', 'Actual 1'],
                    columns=['Predicted 0', 'Predicted 1'])
display(cm_cc_df)

,Predicted 0,Predicted 1
Actual 0,67,34
Actual 1,10150,6954


In [32]:
# Print the imbalanced classification report
print("Imbalanced Classification Report for ClusterCentroids Undersampling")
print(classification_report_imbalanced(y_test, y_pred))

Imbalanced Classification Report for ClusterCentroids Undersampling
                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.66      0.41      0.01      0.52      0.28       101
          1       1.00      0.41      0.66      0.58      0.52      0.26     17104

avg / total       0.99      0.41      0.66      0.57      0.52      0.26     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [33]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

In [34]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [35]:
# Calculated the balanced accuracy score
balanced_acc_score['SMOTEENN'] = balanced_accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy Score : {balanced_acc_score}")

Balanced Accuracy Score : {'Naive Random Oversampling': 0.6661865905954487, 'SMOTE Oversampling': 0.6610112046050256, 'ClusterCentroids Undersampling': 0.5349689494206671, 'SMOTEENN': 0.6642696630514314}


In [36]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_smoteenn_df = pd.DataFrame(cm, index=['Actual 0', 'Actual 1'],
                    columns=['Predicted 0', 'Predicted 1'])
display(cm_smoteenn_df)

,Predicted 0,Predicted 1
Actual 0,76,25
Actual 1,7251,9853


In [37]:
# Print the imbalanced classification report
print("Imbalanced Classification Report for SMOTEENN")
print(classification_report_imbalanced(y_test, y_pred))

Imbalanced Classification Report for SMOTEENN
                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.75      0.58      0.02      0.66      0.44       101
          1       1.00      0.58      0.75      0.73      0.66      0.43     17104

avg / total       0.99      0.58      0.75      0.73      0.66      0.43     17205



In [38]:
balanced_acc_score

{'Naive Random Oversampling': 0.6661865905954487,
 'SMOTE Oversampling': 0.6610112046050256,
 'ClusterCentroids Undersampling': 0.5349689494206671,
 'SMOTEENN': 0.6642696630514314}